<a href="https://colab.research.google.com/github/siwarnasri/MlOps_CustomerSatisfaction/blob/main/1_2_Artifact_Lineage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.2: Artifact Lineage

In this notebook, we'll be taking a look at one of the most interesting features of ML pipelines: automated artifact versioning and tracking.
This gives us a precise idea of the accuracy with which each of our models has been created. It also allows us to cache artifacts so that we can exchange parts of our ML pipelines without having to re-run previous steps.

First, if you have not done so already, run the following cell to install ZenML and it's sklearn integration.

In [1]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
!rm -rf .zen
!zenml init
%pip install pyparsing==2.4.2  # required for Colab

import IPython

# automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.9 MB/s eta 0:00

NumExpr defaulting to 2 threads.
⠴ Installing integrations...
NumExpr defaulting to 2 threads.
Initializing the ZenML global configuration version to 0.44.3
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository a

{'status': 'ok', 'restart': True}

**Colab Note:** On Colab, you need an [ngrok account](https://dashboard.ngrok.com/signup) to view some of the visualizations later. Please set up an account, then set your user token below:

In [2]:
NGROK_TOKEN = "3veC4w6NbwgWUWWfW5kuF_7tagYBcjcYZXissQQhK99"  # TODO: set your ngrok token if you are working on Colab

In [3]:
from zenml.environment import Environment

if Environment.in_google_colab():  # Colab only setup

    # clone zenbytes repo to get source code of previous lessons
    !git clone https://github.com/zenml-io/zenbytes.git  # noqa
    !mv zenbytes/steps .
    !mv zenbytes/pipelines .

    # install and authenticate ngrok
    !pip install pyngrok
    !ngrok authtoken {NGROK_TOKEN}

Cloning into 'zenbytes'...
remote: Enumerating objects: 877, done.
remote: Counting objects: 100% (227/227), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 877 (delta 123), reused 162 (delta 87), pack-reused 650
Receiving objects: 100% (877/877), 16.13 MiB | 25.65 MiB/s, done.
Resolving deltas: 100% (478/478), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=9a1234fecd70a7cfd96241c9304a72a22f5a72d0c132828432c944b72c256945
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Before we get into versioning and caching, let's clarify what exactly **artifacts** are.
To illustrate, let's first rebuild our Digits pipeline from the previous lesson:

In [4]:
from zenml.pipelines import pipeline

from steps.evaluator import evaluator
from steps.importer import importer
from steps.sklearn_trainer import svc_trainer


@pipeline
def digits_pipeline(importer, trainer, evaluator):
    """Links all the steps together in a pipeline"""
    X_train, X_test, y_train, y_test = importer()
    model = trainer(X_train=X_train, y_train=y_train)
    evaluator(X_test=X_test, y_test=y_test, model=model)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


NumExpr defaulting to 2 threads.
The @step decorator that you used to define your evaluatorstep is deprecated. Check out the 0.40.0 migration guide for more information on how to migrate your steps to the new syntax: https://docs.zenml.io/reference/migration-guide/migration-zero-forty
The @step decorator that you used to define your importerstep is deprecated. Check out the 0.40.0 migration guide for more information on how to migrate your steps to the new syntax: https://docs.zenml.io/reference/migration-guide/migration-zero-forty
Using the Output class to define the outputs of your steps is deprecated. You should instead use the standard Python way of type annotating your functions. Check out our documentation https://docs.zenml.io/user-guide/advanced-guide/pipelining-features/configure-steps-pipelines#step-output-names for more information on how to assign custom names to your step outputs.
The @step decorator that you used to define your get_reference_datastep is deprecated. Check 

The artifacts of this pipeline are simply the local variables we have defined: `X_train`, `X_test`, `y_train`, `y_test`, and `model`. These form the data that flows in and out of our steps. ZenML automatically stores, tracks, and version these artifacts.
ZenML stores these artifacts for you so that you can more easily reproduce your ML workflows in the future.
You can reproduce them in the future.

## Pipeline Visualization in ZenML

To see how the steps connect the different artifacts, you can check the
visualization of the pipeline run in the ZenML dashboard, as you learned in the previous
previous lesson.

To do this, run the following code cells again and navigate to the
Pipeline Runs in the ZenML Dashboard.

In [5]:
digits_svc_pipeline = digits_pipeline(
    importer=importer(), trainer=svc_trainer(), evaluator=evaluator()
)
digits_svc_pipeline.run(unlisted=True)

Using the Output class to define the outputs of your steps is deprecated. You should instead use the standard Python way of type annotating your functions. Check out our documentation https://docs.zenml.io/user-guide/advanced-guide/pipelining-features/configure-steps-pipelines#step-output-names for more information on how to assign custom names to your step outputs.
Initiating a new run for the pipeline: digits_pipeline.
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Using the Output class to define the outputs of your steps is deprecated. You should instead use the standard Python way of type annotating your functions. Check out our documentation https://docs.zenml.io/user-guide/advanced-guide/pipelining-features/configure-steps-pipelines#step-output-names for more information on how to assign custom names to your step outputs.
Step importer has started.
Using the Output class to define the outputs of your steps is depre

In [6]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
    if Environment.in_google_colab():
        from pyngrok import ngrok

        public_url = ngrok.connect(port)
        print(f"\x1b[31mIn Colab, use this URL instead: {public_url}!\x1b[0m")
        !zenml up --blocking --port {port}

    else:
        !zenml up --port {port}

start_zenml_dashboard()

INFO:pyngrok.ngrok:Opening tunnel named: http-8237-85962b71-83db-4576-8e5b-ac5ada7df913


Opening tunnel named: http-8237-85962b71-83db-4576-8e5b-ac5ada7df913


INFO:pyngrok.process.ngrok:t=2023-10-01T10:23:28+0000 lvl=info msg="no configuration paths supplied"


t=2023-10-01T10:23:28+0000 lvl=info msg="no configuration paths supplied"


t=2023-10-01T10:23:28+0000 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=/root/.config/ngrok/ngrok.yml legacy_path=/root/.ngrok2/ngrok.yml


...

In Colab, use this URL instead: NgrokTunnel: "https://0f87-34-132-168-98.ngrok-free.app" -> "http://localhost:8237"!
t=2023-10-01T10:23:28+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-85962b71-83db-4576-8e5b-ac5ada7df913 addr=http://localhost:8237 url=https://0f87-34-132-168-98.ngrok-free.app


INFO:pyngrok.process.ngrok:t=2023-10-01T10:23:28+0000 lvl=info msg=end pg=/api/tunnels id=0223f83c49413bb2 status=201 dur=40.7058ms


t=2023-10-01T10:23:28+0000 lvl=info msg=end pg=/api/tunnels id=0223f83c49413bb2 status=201 dur=40.7058ms
NumExpr defaulting to 2 threads.
Deploying a local ZenML server with name 'local'.
Starting ZenML Server as blocking process... press CTRL+C once to stop it.
INFO:     Started server process [1153]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8237 (Press CTRL+C to quit)


INFO:pyngrok.process.ngrok:t=2023-10-01T12:13:45+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


INFO:     Shutting down
t=2023-10-01T12:13:45+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
INFO:     Finished server process [1153]
ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/starlette/routing.py", line 686, in lifespan
    await receive()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/lifespan/on.py", line 137, in receive
    return await self.receive_queue.get()
  File "/usr/lib/python3.10/asyncio/queues.py", line 159, in get
    await getter
asyncio.exceptions.CancelledError



**Note:** If you are running on Colab, you cannot access the regular dashboard link. Use the `ngrok.io` link provided above instead!

You should now see an interactive visualization in the ZenML dashboard, as shown below.
The rectangles represent your pipeline steps and the circles represent your pipeline artifacts.
Also note that the different nodes are color coded so that if your pipeline ever
fails or runs too long, you can find the responsible step at a glance!

<!-- ![Dash Visualization](_assets/1-2/dashboard_initial.png) -->

You should now see an interactive visualization in the ZenML dashboard, as shown below.
The rectangles represent your pipeline steps and the circles represent your pipeline artifacts.
Also note that the different nodes are color coded so that if your pipeline ever
fails or runs too long, you can find the responsible step at a glance!

<!-- ![Dash Visualization](_assets/1-2/dashboard_initial.png) -->

## Caching of artifacts
As mentioned at the outset, tracking which artifact goes to which
steps, allows us to cache and reuse artifacts. Let's look at this in action
By re-running our pipeline without making any changes:

In [7]:
digits_svc_pipeline.run(unlisted=True)

Initiating a new run for the pipeline: digits_pipeline.
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Using the Output class to define the outputs of your steps is deprecated. You should instead use the standard Python way of type annotating your functions. Check out our documentation https://docs.zenml.io/user-guide/advanced-guide/pipelining-features/configure-steps-pipelines#step-output-names for more information on how to assign custom names to your step outputs.
Using cached version of importer.
Step importer has started.
Using cached version of trainer.
Step trainer has started.
Using cached version of evaluator.
Step evaluator has started.
Run digits_pipeline-2023_10_01-12_13_50_063816 has finished in 1.545s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


In the output above, you should now see that each step of the run was cached:
```
Creating unlisted run ... (Caching enabled)
...
Using cached version of importer.
...
Using cached version of svc_trainer.
...
Using cached version of evaluator.
```

In the dashboard you should see the same.
Navigate to the pipeline "Runs" tab again, click on the newest run, and view
its DAG.

In [ ]:
start_zenml_dashboard()

You should now see a visualization as shown below. Note that the icons of all the
steps have changed. This means that they were cached from a previous run.

<!-- ![Visualization of dashboard run cached](_assets/1-2/dashboard_cached.png) -->

Now let's replace the SVC model in our ML pipeline with a decision tree and see what happens.

In [ ]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.tree import DecisionTreeClassifier
from zenml.steps import step


@step()
def tree_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
    """Train an sklearn decision tree classifier."""
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    return model


# redefine and rerun our pipeline, this time with tree_trainer()
digits_tree_pipeline = digits_pipeline(
    importer=importer(), trainer=tree_trainer(), evaluator=evaluator()
)
digits_tree_pipeline.run(unlisted=True)

In the above output you should now see that the `importer`step has still been
has been cached because the underlying data has not changed. However, the trainer and
Evaluator had to be run again, however.

In the dashboard, you should see the same thing again.
Navigate to the Runs pipeline tab again, click on the last run and show
its DAG.

In [ ]:
start_zenml_dashboard()

The visualization should now look like the one shown below. Since we changed the trainer
the corresponding node and all subsequent nodes have been executed and new
artifacts have been created. Note, however, that the artifacts of the dataset are still cached.
They did not need to be recreated.
In an actual production environment, this could save a tremendous amount of time and resources
and resources, as these data artifacts may have resulted from a complex,
expensive pre-processing process.

<!-- ![Dash visualization partially cached](_assets/1-2/dashboard_partly_cached.png) -->

## Artifact storage

You may now be wondering how our ML pipelines can keep track of which artifacts have changed and which have not. This requires several additional MLOps components that you would normally have to set up and configure yourself. Fortunately, ZenML has set this up automatically for us.

Under the hood, all artifacts in our ML pipeline are automatically stored in an [Artifact Store] (https://docs.zenml.io/user-guide/starter-guide/understand-stacks#artifact-store). By default, this is simply a location in your local file system, but we can also configure ZenML to store this data in a cloud bucket like [Amazon S3] (https://docs.zenml.io/component-gallery/artifact-stores/s3) or some other location instead. We will look at this in more detail in a later chapter when we migrate our MLOps stack to the cloud.

## ZenML MLOps Stacks

Artifact stores, together with orchestrators, form the backbone of a ZenML
**Stack** that defines the entire infrastructure and tools on which your ML
workflows are executed.

<!-- ![Local MLOps stack](_assets/1-2/local_stack_redesigned.png) -->

If you have enabled the ZenML dashboard, you will see a list of all MLOps
stacks under the "Stacks" section. Currently, you will only see the "default" stack there.
Stack, which consists of a local artifact store and a local orchestrator.

Under the "Stack Components" tab, you can browse all the stack components that you have
you currently have registered with ZenML. You can combine these as you see fit
combine them to form new stacks. Currently, you should only have a single "default" component for
"Orchestrator" and "Artifact Store", but we will be registering additional
Stack components in later lessons.

<!-- ![Dashboard Stack List](_assets/1-2/dashboard_stack_list.png) -->

If you click on the "Default" artifact store and navigate to the "Runs" tab,
you will see all the runs that have been executed for that component.

<!-- ![Dashboard Stack List](_assets/1-2/dashboard_artifact_store_run_list.png) -->

In the following steps, we will add more components to our MLOps stack, including model deployment tools, experiment trackers, data and model monitoring tools, and more. Let's start with experiment tracking in the [next notebook](2-1_Experiment_Tracking.ipynb).